In [38]:
%load_ext autoreload
%autoreload 2

In [41]:
from evo.dataset import FastaDataset

pfam_path  = "/homefs/home/lux70/storage/data/pfam/Pfam-A.fasta"
ds = FastaDataset(pfam_path)


100%|██████████| 10.6G/10.6G [01:36<00:00, 118MB/s] 
100%|██████████| 10.6G/10.6G [01:15<00:00, 151MB/s]


In [42]:
dl = torch.utils.data.DataLoader(ds, batch_size=256)
batch = next(iter(dl))

In [ ]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis

import pandas as pd

df = pd.DataFrame({"sequences": seqs})
df = df.iloc[:10, ]

def _protein_property(protein_sequence, property):
    analyzer = ProteinAnalysis(protein_sequence)
    if property == "charge_at_pH":
        return getattr(analyzer, property)(pH=7)
    else:
        return getattr(analyzer, property)()

properties = ['molecular_weight', 'aromaticity', 'instability_index', 'isoelectric_point', 'gravy', 'charge_at_pH']


for property in properties:
    df[property] = df.sequences.map(lambda seq: _protein_property(seq, property))


In [64]:
N = 0

target_N = 1000
all_dfs = []

while N < target_N:
    batch = next(iter(dl))
    seqs = batch[1]
    seqs = [s.replace("X", "") for s in seqs]
    df = pd.DataFrame({"sequences": seqs})
    for property in properties:
        df[property] = df.sequences.map(lambda seq: _protein_property(seq, property))
    all_dfs.append(df)

df = pd.concat(all_dfs, axis=0)


KeyboardInterrupt: 

In [65]:
import os
os.cpu_count()

96

,sequences,molecular_weight,aromaticity,instability_index,isoelectric_point,gravy,charge_at_pH
0,AFQYVQANPDEVCPANWTPGEKTMKPDPEGSKEYFSA,4133.4840,0.135135,7.005405,4.359289,-0.983784,-3.205538
1,GLQYNRKTGEVVPANWVPGQSGITRDTKYIGR,3561.9579,0.093750,31.053125,9.988812,-0.834375,2.760007
2,AAQYVRTHPGEVCPAKWKEDGETLKPSFDLVGK,3658.1018,0.090909,19.051515,6.799899,-0.721212,-0.121184
3,AFQYATAHPGEVCPANWKPGHKTIKADQDAKYDFFDA,4139.5182,0.162162,29.264865,6.018590,-0.764865,-1.038379
4,AAQYTAAHPNEVCPAKWKEGEKTLKPSLDLVGK,3581.0609,0.060606,6.615152,8.168870,-0.678788,0.876706
5,AFQYVETHGEVCPANWTPDSPTIKPSPEGSKEYFEK,4070.4051,0.138889,39.688889,4.699243,-0.975000,-3.116674
6,AFQFADKHGEVCPANWNPDTNADTIKPNPKGSKEYFQK,4294.6712,0.131579,-14.826316,6.794385,-1.210526,-0.123863
7,AFQYTDEHGEVCPAGWKPGSDTIKPNVDDSKEYFSK,4047.3286,0.138889,2.780556,4.647406,-1.083333,-3.119932
8,ALQFHEEHGEVCPAQWAKDKAGMDASPEGVAKYLAS,3872.2572,0.083333,39.869444,4.969399,-0.513889,-3.030191
9,SLQTGDKHKVTTPINWVPGDDVIVHPSVKTEQAKEIFPE,4341.8289,0.051282,41.064103,5.482826,-0.592308,-2.352470


In [14]:
log_to_wandb = False
shape = (16, 64, 8)

# sample latent (implicitly uncompresses and unnormalizes)
print("sampling latent...")
x, log_dict = callback.sample_latent(shape)   # compressed, i.e. (N, L, C_compressed)

# # uncompress and unnormalize the sampled x
latent = callback.diffusion.process_x_to_latent(x)  # (N, L, C_comp) -> (N, L, C_lat)

if callback.calc_sequence:
    seq_str, log_dict = callback.construct_sequence(latent, device)

sampling latent...


  0%|                                                                                                                                            | 0/1 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:29<00:00, 29.87s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Mean perplexity: 24.415


In [35]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
analyzer = ProteinAnalysis(seq_str[0])
import numpy as np

In [37]:
for k, v in results.items():
    print(k, v)

molecular_weight 7608.8577
aromaticity 0.125
instability_index 61.553281250000005
isoelectric_point 9.339873313903809
gravy -0.27187500000000003
flexibility 0.9885220779220779
charge_at_pH 5.046701083983767
sec_struct_frac (0.234375, 0.203125, 0.34375)
molar_extinction_coefficient (15470, 15720)


In [27]:
s = "gravy"
unscaler = LatentScaler()
getattr(analyzer, s)()

-0.27187500000000003

In [ ]:
properties 